In [3]:
# Ensure necessary library is installed
install.packages("rvest")
install.packages("tidyverse")
install.packages("lubridate")

Installing package into ‘/srv/rlibs’
(as ‘lib’ is unspecified)

Installing package into ‘/srv/rlibs’
(as ‘lib’ is unspecified)

Installing package into ‘/srv/rlibs’
(as ‘lib’ is unspecified)



In [4]:
# Loading Libraries
library(rvest)
library(tidyverse)
library(lubridate)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()         masks stats::filter()
✖ readr::guess_encoding() masks rvest::guess_encoding()
✖ dplyr::lag()            masks stats::lag()


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [5]:
#Scrape Data from the Web Page
url <- "https://rateyourmusic.com/charts/top/album/all-time/"
webpage <- read_html(url)

In [6]:
# Extract CSS Selectors
album_names <- webpage %>% 
  html_nodes(".page_charts_section_charts_item_title") %>% 
  html_text() %>%
  str_trim() # Remove any leading or trailing white spaces
artists <- webpage %>% 
  html_nodes(".page_charts_section_charts_item_credited_links_primary") %>% 
  html_text() %>%
  str_trim() # Remove any leading or trailing white spaces
ratings <- webpage %>%
  html_nodes(".page_charts_section_charts_item_details_average_num") %>%
  html_text(trim = TRUE) %>%
  as.numeric() # convert to numeric
ratings <- ratings[seq(1, length(ratings), 2)]
release_date <- webpage %>% 
  html_nodes(".page_charts_section_charts_item_date span:first-child") %>% 
  html_text() %>%
  str_trim() %>% # Remove any leading or trailing white spaces
  dmy() %>%  # parse as day-month-year
  format("%d-%m-%Y")  # format as dd-mm-yyyy
genres <- webpage %>% 
  html_nodes(".page_charts_section_charts_item_genres_primary") %>% 
  map_chr(~.x %>% # This function from the purrr package is used to apply the nested function to each node, and ensures that the results are returned as character vectors.
            html_nodes("a") %>% 
            html_text() %>% 
            paste(collapse = ", ") #This results in a single string for each album that lists all associated genres in one go.
  )
num_ratings <- webpage %>% 
  html_nodes(".page_charts_section_charts_item_details_ratings .full") %>% 
  html_text() %>%
  str_trim() %>% # Remove any leading or trailing white spaces
  str_remove_all(",") %>%  # remove commas from numbers
  as.numeric() # convert to numeric
num_views <- webpage %>% 
  html_nodes(".page_charts_section_charts_item_details_reviews .full") %>% 
  html_text() %>%
  str_trim() %>%
  as.numeric() # convert to numeric
descriptors <- webpage %>% 
  html_nodes(".page_charts_section_charts_item_genre_descriptors") %>% 
  html_text(trim = TRUE) %>%
  str_replace_all("\n", ", ") %>%
  str_squish()


Warning message:
“ 2 failed to parse.”
Warning message in webpage %>% html_nodes(".page_charts_section_charts_item_details_reviews .full") %>% :
“NAs introduced by coercion”


In [7]:
# Transform into a data frame
df <- data.frame(
  Album = album_names,
  Artist = artists,
  Rating = ratings,
  ReleaseDate = release_date,
  Genres = genres,
  Descriptors = descriptors,
  NumberOfRatings = num_ratings,
  NumberOfViews = num_views
)
# Adding a ranking column
df <- df %>% mutate(Ranking = row_number())
# To ensure the Ranking column is first
df <- df %>% select(Ranking, everything())
head(df)


,Ranking,Album,Artist,Rating,ReleaseDate,Genres,Descriptors,NumberOfRatings,NumberOfViews
,<int>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,1,To Pimp a Butterfly,Kendrick Lamar,4.36,15-03-2015,"Conscious Hip Hop, West Coast Hip Hop, Jazz Rap","political, conscious, concept album, poetic, introspective, urban, protest, eclectic",69640,597
2,2,OK Computer,Radiohead,4.27,16-06-1997,"Alternative Rock, Art Rock","melancholic, anxious, alienation, futuristic, existential, lonely, atmospheric, cold",94735,NA
3,3,Wish You Were Here,Pink Floyd,4.32,12-09-1975,"Progressive Rock, Art Rock","melancholic, atmospheric, progressive, concept album, serious, longing, introspective, alienation",65282,997
4,4,Madvillainy,Madvillain,4.33,23-03-2004,Abstract Hip Hop,"sampling, playful, abstract, humorous, cryptic, mysterious, eclectic, surreal",54310,469
5,5,In the Court of the Crimson King,King Crimson,4.31,10-10-1969,"Progressive Rock, Art Rock","fantasy, epic, progressive, complex, poetic, surreal, philosophical, melancholic",59998,939
6,6,In Rainbows,Radiohead,4.29,10-10-2007,"Art Rock, Alternative Rock","lush, melancholic, introspective, bittersweet, atmospheric, mellow, warm, ethereal",68581,872
